# **Synthetic error from catalog**

A demonstration notebook that allows generating photometric errors randomly distributed over the observed magnitudes, from empirical probability functions created using a catalog of observations.

## 1. **Initialization**

### 1.1 Import packages

In [ ]:
from importlib import reload
import os
import sys
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from astropy.io import fits
from astropy.wcs import WCS

sys.path.append(os.path.abspath(os.path.join(os.getcwd(),"../")))

from src import photometry_plots as pp
from src import photometry_errors as pe; reload(pe)
# %matplotlib widget
%matplotlib inline

### 1.2 Load source catalog

In [ ]:
base_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))

In [ ]:
### Example with LSST DP1 ECDFS ###
dp1_ECDFS = pd.read_csv(f"{base_dir}/data/DP1_ECDFS_photometry.csv")
# dp1_ECDFS

### 1.3 Load maglim maps

Corresponding bands must be specified 

In [ ]:
### Retrieve magmap list ###
mag_lim_folder = os.path.join(base_dir, 'data/maglim_maps')
mag_maps_file_list = [os.path.join(mag_lim_folder,f) for f in os.listdir(mag_lim_folder) 
                        if f.endswith('.fits')]

### Define band list ###
band_list = ["u","g","r","i","z","y"]

### Build magmap dict ###
mag_maps_file_dict = {}
for band, f in zip(band_list, mag_maps_file_list):
    try:
        if f.endswith('.fits'):
            mag_maps_file_dict[band] = f
    except:
        continue

mag_maps_file_dict

In [ ]:
# Initialisation
maglim = pe.DeltaMaps(mag_maps_file_dict)

# see avalaible bands
maglim.bands()

# plot one band
maglim.plot_map("g")

## 2. **Compute $\Delta_{band,i} = mag_{band,i}-mag\_lim_{band,radec(i)}$**

### 2.1 Compute $\Delta$ for each source

In [ ]:
# mag columns dict
mag_columns = {
    "u": "u_cModelMag",
    "g": "g_cModelMag",
    "r": "r_cModelMag",
    "i": "i_cModelMag",
    "z": "z_cModelMag",
    "y": "y_cModelMag",
}

# add delta to catalog
cat = maglim.add_delta_to_catalog(dp1_ECDFS, mag_columns, radec_names=["coord_ra","coord_dec"])


In [ ]:
list(cat.columns.values)


In [ ]:
cat#[['y_cModelMag', 'y_cModelMagErr', 'maglim_y', 'delta_y']]

## 3. **Generate probability distribution functions** 

In [ ]:
cat = cat[(cat["delta_r"]<-1)&(cat["delta_g"]<-1)&(cat["delta_i"]<-1)]

In [ ]:
pem = pe.PhotometricErrorModel()

models = pem.model_collection(x_lists=cat[["delta_r","delta_i","delta_z"]], 
    sigma_lists=cat[["r_cModelMagErr", "i_cModelMagErr", "z_cModelMagErr"]], 
    model_names=["r", "i", "z"])


In [ ]:
# %matplotlib widget
pem.plot_models(
    models,
    x_lists=cat[["delta_r","delta_i","delta_z"]],
    sigma_lists=cat[["r_cModelMagErr", "i_cModelMagErr", "z_cModelMagErr"]],
    model_names=["r", "i", "z"],
    show_scatter=True,
    show_model=True,
    subsample=20000
)


In [ ]:
sigma_sim_r = models["i"].sample(cat["delta_i"])

In [ ]:
mask=cat['delta_r']<1e20
plt.figure()
plt.scatter(cat['delta_i'],sigma_sim_r, s=2, alpha=0.2)
plt.scatter(cat['delta_i'][mask],cat['i_cModelMagErr'][mask], s=2, alpha=0.2)
plt.show()

## 4. **Test: Attribute LSST DP1-like errors to a catalog**